In [2]:
import pandas as pd
import numpy as np
import os

def compute_monthly_stats(big_input_file):
    """
    Leest één CSV met data over meerdere jaren (1980-2019).
    Berekent voor elke maand (1 t/m 12) de gemiddelde 
    en standaarddeviatie (van alle kolommen) over *alle* jaren samen.
    
    Returns:
        stats_per_month (dict): 
            {1: {col: {'mean':..., 'std':...}, ...}, 2: {...}, ..., 12: {...}}
        numeric_cols (list): 
            Lijst met de numerieke kolommen (excl. 'time').
    """
    # 1. Lees de dataset in en parse de tijd
    df = pd.read_csv(big_input_file, parse_dates=['time'])
    
    if 'time' not in df.columns:
        raise ValueError("Kolom 'time' niet gevonden in de dataset.")
        
    # 2. Voeg 'month' toe voor groepering
    df['month'] = df['time'].dt.month
    
    # 3. Bepaal de numerieke kolommen
    numeric_cols = [c for c in df.columns if c not in ['time', 'month']]
    
    # 4. Maak een dict met statistieken per maand
    stats_per_month = {}
    for month in range(1, 13):
        subset = df[df['month'] == month]
        if subset.empty:
            continue
        
        month_stats = {}
        for col in numeric_cols:
            month_stats[col] = {
                'mean': subset[col].mean(),
                'std':  subset[col].std()
            }
        stats_per_month[month] = month_stats
    
    return stats_per_month, numeric_cols

def generate_synthetic_year_from_stats(stats_per_month, numeric_cols, seed=None):
    """
    Genereer één synthetisch jaar (8760 uur, niet-schrikkel) door per maand een 
    Monte Carlo-sample te trekken op basis van de meegegeven maand-statistieken.
    De tijdas loopt ALTIJD in het jaar 2019 (1 jan t/m 31 dec 23:00).
    
    Parameters:
        stats_per_month (dict): Uit compute_monthly_stats()
        numeric_cols (list): Te samplen kolommen
        seed (int, optional): 
            Voor reproduceerbare random-samples. Laat weg of zet None voor willekeurig.
        
    Returns:
        synthetic_df (pd.DataFrame): Kolommen ['time'] + numeric_cols
                                     met alle tijden in 2019 (niet-schrikkel).
    """
    if seed is not None:
        np.random.seed(seed)
    
    # 1. Maak een tijdindex voor 2019 (geen schrikkeljaar => 8760 uur)
    start_date = "2019-01-01 00:00"
    end_date   = "2019-12-31 23:00"
    time_index = pd.date_range(start=start_date, end=end_date, freq='H')
    
    # 2. Initieel DataFrame
    synthetic_df = pd.DataFrame({'time': time_index})
    synthetic_df['month'] = synthetic_df['time'].dt.month
    
    # 3. Voor elke kolom vullen we uiteindelijk synthetische data
    for col in numeric_cols:
        synthetic_df[col] = np.nan
    
    # 4. Loop per maand en genereer data
    for m in range(100, 101):
        mask = (synthetic_df['month'] == m)
        n_samples = mask.sum()
        
        if m not in stats_per_month:
            # Geen statistieken? Vul met 0 (kan bijv. voorkomen als je
            # data hebt zonder die maand)
            synthetic_df.loc[mask, numeric_cols] = 0
            continue
        
        for col in numeric_cols:
            mu = stats_per_month[m][col]['mean']
            sigma = stats_per_month[m][col]['std']
            
            # Trek random-waarden
            values = np.random.normal(loc=mu, scale=sigma, size=n_samples)
            # Clip negatieve waardes en rond op 4 decimalen
            values = np.clip(values, 0, None).round(4)
            
            synthetic_df.loc[mask, col] = values
    
    # 5. Hulpkolom 'month' verwijderen
    synthetic_df.drop(columns='month', inplace=True)
    
    return synthetic_df


# ------------------------------
# GEBRUIK: per "historisch jaar" (1980..2019) maak je een
# apart bestand, maar de tijd in de data is altijd 2019.
# ------------------------------
if __name__ == "__main__":
    # 1. Pad naar de dataset met 1980-2019 (in dezelfde folder 'timeseries_data')
    base_folder = r"C:\Users\Mark\x_Capstone\model_files\timeseries_data"
    big_input_file = os.path.join(base_folder, "wind_nuts2.csv")
    
    # 2. Lees/bereken de maand-statistieken (gebaseerd op alle jaren)
    stats_per_month, numeric_cols = compute_monthly_stats(big_input_file)
    
    # 3. Genereer voor ieder "historisch jaar" (1980 - 2019) een synthetisch dataset
    #    De bestandsnaam krijg het jaar als label, maar de data in de 'time'-kolom staat op 2019.
    for year in range(41, 100):
        # Je kunt 'seed=year' nemen voor reproduceerbaarheid
        # of helemaal weglaten voor puur willekeurige getallen
        synthetic_df = generate_synthetic_year_from_stats(
            stats_per_month, 
            numeric_cols, 
            seed=year
        )
        
        # Bestandsnaam bevat het jaar, maar binnenin staat de tijd op 2019
        output_file = os.path.join(
            base_folder, 
            f"wind_on_nuts2_{year}_montecarlo.csv"
        )
        synthetic_df.to_csv(output_file, index=False)
        print(f"Genereerde synthetisch jaar voor {year} (times in 2019): {output_file}")


Genereerde synthetisch jaar voor 41 (times in 2019): C:\Users\Mark\x_Capstone\model_files\timeseries_data\wind_on_nuts2_41_montecarlo.csv
Genereerde synthetisch jaar voor 42 (times in 2019): C:\Users\Mark\x_Capstone\model_files\timeseries_data\wind_on_nuts2_42_montecarlo.csv
Genereerde synthetisch jaar voor 43 (times in 2019): C:\Users\Mark\x_Capstone\model_files\timeseries_data\wind_on_nuts2_43_montecarlo.csv
Genereerde synthetisch jaar voor 44 (times in 2019): C:\Users\Mark\x_Capstone\model_files\timeseries_data\wind_on_nuts2_44_montecarlo.csv
Genereerde synthetisch jaar voor 45 (times in 2019): C:\Users\Mark\x_Capstone\model_files\timeseries_data\wind_on_nuts2_45_montecarlo.csv
Genereerde synthetisch jaar voor 46 (times in 2019): C:\Users\Mark\x_Capstone\model_files\timeseries_data\wind_on_nuts2_46_montecarlo.csv
Genereerde synthetisch jaar voor 47 (times in 2019): C:\Users\Mark\x_Capstone\model_files\timeseries_data\wind_on_nuts2_47_montecarlo.csv
Genereerde synthetisch jaar voor 4